<a href="https://colab.research.google.com/github/Iremguel/Fallstudie_Elektrifizierung_der_Logistik/blob/main/Stufe_1_(Nur_Diesel).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie Elektrifizierung

In [56]:
! pip install -q pyscipopt

In [57]:
import pandas as pd
from pyscipopt import Model, quicksum

In [58]:
! git clone https://github.com/Iremguel/Fallstudie_Elektrifizierung_der_Logistik.git

fatal: destination path 'Fallstudie_Elektrifizierung_der_Logistik' already exists and is not an empty directory.


In [59]:
! ls

Fallstudie_Elektrifizierung_der_Logistik  sample_data


In [60]:
! ls Daten

ls: cannot access 'Daten': No such file or directory


In [61]:
folder = "Fallstudie_Elektrifizierung_der_Logistik/Daten/"

In [62]:
chargers = pd.read_csv(f"{folder}chargers.csv", sep=";")

In [63]:
diesel_trucks = pd.read_csv(f"{folder}/diesel_trucks.csv", sep=";")

In [64]:
electric_trucks = pd.read_csv(f"{folder}/electric_trucks.csv", sep=";")

In [65]:
routes = pd.read_csv(f"{folder}/routes.csv", sep=";")

## Hilfunktionen

In [66]:
def time_to_slot(t):
    """Wandelt HH:MM in 15-Minuten-Zeitschritte um"""
    h, m = map(int, t.split(":"))
    return (60 * h + m) // 15

routes["start"] = routes["starttime"].apply(time_to_slot)
routes["end"]   = routes["endtime"].apply(time_to_slot)

## Indexmengen

In [67]:
R = list(routes["route_id"].unique()) # Routen

#LKWs
K_D = list(diesel_trucks["truck_model"].unique()) # Diesel LKW-Typen
K_E = list(electric_trucks["truck_model"].unique()) # e-LKW Typen
K = K_D + K_E # Gesamte LKW-Flotte

V = list(range(len(R)))      # physische LKWs (Ausprägung)

S = list(chargers["charger_model"].unique()) # Ladesäulentypen

T = list(range(96))  # 24h * 4

In [68]:
DAYS = list(range(260))

## Parameter

In [69]:
# Diesel-LKWs
#diesel_capex = diesel_trucks.set_index("truck_model")["capex_yearly"].to_dict()
#diesel_opex  = diesel_trucks.set_index("truck_model")["opex_yearly"].to_dict()
#diesel_kfz   = diesel_trucks.set_index("truck_model")["kfz_yearly"].to_dict()

#diesel_consumption = diesel_trucks.set_index("truck_model")["avg_diesel_per_100km"].to_dict()

In [70]:
# Diesel-LKW-Parameter (nur ein Typ)
truck = "ActrosL"

diesel_capex = diesel_trucks.set_index("truck_model").loc[truck, "capex_yearly"]
diesel_opex  = diesel_trucks.set_index("truck_model").loc[truck, "opex_yearly"]
diesel_kfz   = diesel_trucks.set_index("truck_model").loc[truck, "kfz_yearly"]
diesel_cons  = diesel_trucks.set_index("truck_model").loc[truck, "avg_diesel_per_100km"]

# Konstanten
diesel_price = 1.60  # €/Liter
toll_price   = 0.34  # €/km
DAYS = 260           # Betriebstage pro Jahr

# Fixkosten pro LKW (jährlich)
truck_fixed_cost = diesel_capex + diesel_opex + diesel_kfz

In [71]:
# Tourenparameter
route_start = routes.set_index("route_id")["start"].to_dict()
route_end   = routes.set_index("route_id")["end"].to_dict()
distance_total = routes.set_index("route_id")["distance_total"].to_dict()
distance_toll  = routes.set_index("route_id")["distance_toll"].to_dict()

In [72]:
# Routenkosten pro Jahr
route_cost = {}
for r in R:
    fuel_cost = diesel_cons / 100 * distance_total[r] * diesel_price
    toll_cost = distance_toll[r] * toll_price
    route_cost[r] = DAYS * (fuel_cost + toll_cost)

In [73]:
#Ladesäulen
charger_capex = chargers.set_index("charger_model")["capex_yearly"].to_dict()
charger_opex  = chargers.set_index("charger_model")["opex_yearly"].to_dict()
charger_power = chargers.set_index("charger_model")["max_power"].to_dict()
charger_spots = chargers.set_index("charger_model")["charging_spots"].to_dict()

In [74]:
# e-LKWs
# Kosten und Erlöse
electric_capex = electric_trucks.set_index("truck_model")["capex_yearly"].to_dict()
electric_opex  = electric_trucks.set_index("truck_model")["opex_yearly"].to_dict()
thg_revenue    = electric_trucks.set_index("truck_model")["thg_yearly"].to_dict()

# Energie und Technik
electric_consumption = electric_trucks.set_index("truck_model")["avg_energy_kWh_per_100km"].to_dict()
truck_max_power      = electric_trucks.set_index("truck_model")["max_power"].to_dict()
soc_max              = electric_trucks.set_index("truck_model")["soc_max_kWh"].to_dict()


In [75]:
route_slots = {}
for r in R:
    start_slot = route_start[r] // 15
    end_slot   = route_end[r] // 15
    route_slots[r] = list(range(start_slot, end_slot))


In [76]:
# Diesel-Kosten pro Route & LKW
#diesel_cost = {}
#for r in R:
#    for k in K_D:
#        fuel_cost = (
#            diesel_consumption[k] / 100
#            * distance_total[r]
#            * diesel_price
#        )
#        toll_cost = distance_toll[r] * toll_price
#        diesel_cost[r, k] = fuel_cost + toll_cost


In [77]:
# Energiebedarf E-LKW pro Route
energy_need = {}
for r in R:
    for k in K_E:
        energy_need[r, k] = (
            electric_consumption[k] / 100
            * distance_total[r]
        )


In [78]:
# Energiepreise & Netz
energy_price = 0.25      # €/kWh
grid_fee     = 1000      # €/year
power_price  = 150       # €/kW/year

grid_base_capacity = 500
grid_extension     = 500
grid_extension_cost = 10000


In [79]:
# Batteriespeicher-Parameter
batt_capex_power = 30    # €/kW/year
batt_capex_energy = 350  # €/kWh/year
batt_opex_rate = 0.02
battery_efficiency = 0.98
battery_dod = 0.975

## Modellinstanz ziehen

In [80]:
model = Model()

## Entscheidungsvariablen

In [81]:
# Tour–LKW-Zuordnung
# x[r,v] = 1 wenn Route r von LKW v gefahren wird
X = {}
for r in R:
    for v in V:
        X[r, v] = model.addVar(vtype="B", name=f"x_{r}_{v}")

In [82]:
# y[v] = 1 wenn LKW v genutzt wird
Y = {}
for v in V:
    Y[v] = model.addVar(vtype="B", name=f"y_{v}")

In [83]:
# Installation von Ladesäulen
#Y = {}
#for s in S:
#    Y[s] = model.addVar(vtype="B", name=f"y_{s}")

In [84]:
# Erweiterung durch Transformator
#Z = model.addVar(vtype="B", name="grid_extension")

In [85]:
# Batteriespeicher
#P_batt = model.addVar(lb=0, name="battery_power")     # kW
#E_batt = model.addVar(lb=0, name="battery_capacity") # kWh

## Flussvariablen

In [86]:
# Ladeleistung von E-LKW k im Zeitslot t
#p_charge_truck = {}
#for k in K_E:
#    for t in T:
#        p_charge_truck[k, t] = model.addVar(
#            lb=0,
#            name=f"p_charge_{k}_{t}"
#        )

In [87]:
# Laden des Batteriespeichers
#p_batt_charge = {}
#for t in T:
#    p_batt_charge[t] = model.addVar(
#        lb=0,
#        name=f"p_batt_charge_{t}"
#    )

In [88]:
# Entladen des Batteriespeichers
#p_batt_discharge = {}
#for t in T:
#    p_batt_discharge[t] = model.addVar(
#        lb=0,
#        name=f"p_batt_discharge_{t}"
#    )

In [89]:
# Netzbezugsleistung im Slot t
#p_grid = {}
#for t in T:
#    p_grid[t] = model.addVar(lb=0, name=f"p_grid_{t}")

In [90]:
# Optional: Energieverbrauch von Route r durch LKW k
#energy_need[r,k]

## Zielfunktion

In [91]:
model.setObjective(
    # Fixkosten der eingesetzten LKWs
    quicksum(truck_fixed_cost * Y[v] for v in V)
    +
    # variable Routenkosten
    quicksum(route_cost[r] * X[r, v] for r in R for v in V),
    sense="minimize"
)

## Nebenbedingungen

In [92]:
# Jede Route wird genau einmal gefahren
for r in R:
    model.addCons(
        quicksum(X[r, v] for v in V) == 1,
        name=f"route_coverage_{r}"
    )


In [93]:
# Route nur wenn LKW existiert
for r in R:
    for v in V:
        model.addCons(
            X[r, v] <= Y[v],
            name=f"use_truck_{r}_{v}"
        )

In [94]:
# Keine zeitliche Überlappung auf demselben LKW
for v in V:
    for i, r1 in enumerate(R):
        for r2 in R[i+1:]:
            if not (
                route_end[r1] <= route_start[r2]
                or route_end[r2] <= route_start[r1]
            ):
                model.addCons(
                    X[r1, v] + X[r2, v] <= 1,
                    name=f"overlap_{r1}_{r2}_{v}"
                )

In [95]:
# Ladeverbot während der Route
#for r in R:
#    for k in K_E:
#        for t in route_slots[r]:
#            model.addCons(
#                p_charge_truck[k, t] <= (1 - X[r, k]) * truck_max_power[k],
#                name=f"no_charge_during_route_{r}_{k}_{t}"
#            )

In [96]:
# Energiebedarf der Route muss geladen werden
#for r in R:
#    for k in K_E:
#        model.addCons(
#            quicksum(p_charge_truck[k, t] * 0.25 for t in T)
#            >= energy_need[r, k] * X[r, k],
#            name=f"energy_balance_{r}_{k}"
#        )

In [97]:
# Maximale Ladeleistung je e-LKW
#for k in K_E:
#    for t in T:
#        model.addCons(
#            p_charge_truck[k, t] <= truck_max_power[k],
#            name=f"truck_power_limit_{k}_{t}"
#        )


In [98]:
# Ladesäulen-Leistung nicht überschreiten
#for t in T:
#    model.addCons(
#        quicksum(p_charge_truck[k, t] for k in K_E)
#        <= quicksum(Y[s] * charger_power[s] for s in S),
#        name=f"charger_power_limit_{t}"
#    )

In [99]:
# Maximal 3 Ladesäulen
#model.addCons(
#    quicksum(Y[s] for s in S) <= 3,
#    name="max_chargers"
#)

In [100]:
# Netzanschlussleistung mit Transformator
#for t in T:
#    model.addCons(
#        p_grid[t] <= grid_base_capacity + Z * grid_extension,
#        name=f"grid_capacity_{t}"
#    )

In [101]:
# Leistungsbilanz (Netz + Batterie + Laden)
#for t in T:
#    model.addCons(
#        p_grid[t] + p_batt_discharge[t]
#        >= quicksum(p_charge_truck[k, t] for k in K_E)
#        + p_batt_charge[t],
#        name=f"power_balance_{t}"
#    )

In [102]:
# Batteriegrenzen
#for t in T:
#    model.addCons(p_batt_charge[t] <= P_batt)
#    model.addCons(p_batt_discharge[t] <= P_batt)

## Lösung

In [103]:
#model.redirectOutput()

In [104]:
model.optimize()

In [105]:
print("\n==============================")
print("OPTIMALE LÖSUNG (DIESEL)")
print("==============================")
print(f"Gesamtkosten pro Jahr: {model.getObjVal():,.2f} €")

print("\nEingesetzte LKWs:")
used_trucks = [v for v in V if model.getVal(Y[v]) > 0.5]
print(f"Anzahl Diesel-LKWs: {len(used_trucks)}")

print("\nRoutenplanung:")
for r in R:
    for v in V:
        if model.getVal(X[r, v]) > 0.5:
            print(f"Route {r} → LKW {v}")


OPTIMALE LÖSUNG (DIESEL)
Gesamtkosten pro Jahr: 970,159.60 €

Eingesetzte LKWs:
Anzahl Diesel-LKWs: 14

Routenplanung:
Route t-4 → LKW 0
Route t-5 → LKW 1
Route t-6 → LKW 2
Route s-1 → LKW 3
Route s-2 → LKW 4
Route s-3 → LKW 5
Route s-4 → LKW 6
Route w1 → LKW 7
Route w2 → LKW 8
Route w3 → LKW 9
Route w4 → LKW 10
Route w5 → LKW 11
Route w6 → LKW 12
Route w7 → LKW 13
Route r1 → LKW 0
Route r2 → LKW 2
Route r3 → LKW 1
Route h3 → LKW 3
Route h4 → LKW 4
Route k1 → LKW 5
